In [37]:
# Necessary imports
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import glob
import re
from IPython.core.display import display, HTML
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation

print('Libraries imported!')

Libraries imported!


In [2]:
path = r'C:/Metis_Bootcamp/Cocktails_recommendation_NLP/data/main_reviews' # data path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    data = pd.read_csv(filename, index_col=0, header=0)
    li.append(data)

df = pd.concat(li, axis=0, ignore_index=True)

print(df.shape)

df.head()

(10522, 17)


,Name,Description,Category,Country,Alcohol_Vol,Review,Style,Aroma,Flavor,Sweetness,Bitterness,Enjoy,Pairing,Cocktails,Bottom_Line,Points,Price
0,Mezcal de Leyendas Verde San Luis Potosi Mezca...,Mezcal de Leyendas Review Verde San Luis Potos...,Blanco Mezcal,Mexico,45%,"Clear color. Aromas and flavors of cucumber, m...","Fruity, Funky, Rich, Spicy, Smoky, Herbal & Co...","cucumber-melon, lime spritz, caramelized orang...","Same as aromas with notes of pickled ginger, s...",NaN,NaN,"in cocktails, on the rocks and neat",NaN,"Paloma, Sangrita, Tommy's Margarita","A sweet, savory, salty, juicy display of compl...",97 Points,$49.99
1,El Tesoro Paradiso Tequila Extra Anejo Mexico ...,El Tesoro Review Paradiso Tequila Extra Anejo ...,Extra Añejo Tequila,Mexico,40%,"Gold color. Inviting aromas of roasted nuts, c...","Fruity, Complex, Candied, Herbal, Rich & Spicy","roasted nuts, candied cherries, vanilla bean, ...","ripe melons, toffee, and white pepper",NaN,NaN,"in cocktails, neat, on the rocks and with cigars",NaN,"Paloma, Sangrita, Tommy's Margarita",A tantalizing melange of complex flavors that ...,97 Points,$180
2,Peloton de la Muerte Vegan Pechuga Mezcal Mexi...,Peloton de la Muerte Review Vegan Pechuga Mezc...,Flavored Mezcal,Mexico,45.10%,Silvery straw color. Citrusy aromas and flavor...,"Complex, Fruity & Herbal","pickled melon and pineapple, candied jalapeño,...",Same as aromas with suggestions of curry,NaN,NaN,"in cocktails, neat and on the rocks",NaN,"Paloma, Sangrita, Tommy's Margarita","A joyous, extroverted Mezcal with juicy fruit ...",96 Points,$44.99
3,El Tesoro Anejo Tequila Mexico Spirits Review ...,El Tesoro Review Anejo Tequila Mexico Mexican ...,Añejo Tequila,Mexico,40%,Straw color. Inviting aromas and flavors of ho...,"Fruity, Spicy & Herbal","honey-drizzled grapefruit, sweet apple compote...","Same as aromas with notes of pink peppercorns,...",NaN,NaN,"in cocktails, neat and on the rocks",NaN,"Paloma, Sangrita, Tommy's Margarita","A stylish, peppery tequila with an authentic v...",96 Points,$85
4,1800 Milenio Extra Anejo Tequila Mexico Spirit...,1800 Review Milenio Extra Anejo Tequila Mexico...,Extra Añejo Tequila,Mexico,40%,"Amber color. Aromas of toffee coated popcorn, ...","Fruity, Rich & Spicy","toffee coated popcorn, peanuts, warm madeira c...","allspice, mint tea, candied ginger, honey, and...",NaN,NaN,neat and on the rocks,NaN,NaN,A richly flavored Extra Añejo Tequila with an ...,96 Points,$250


In [3]:
#df.to_csv('main_reviews.csv')

In [149]:
df = pd.read_csv('../data/main_reviews.csv', index_col=0)
df.head()

,Name,Description,Category,Country,Alcohol_Vol,Review,Style,Aroma,Flavor,Sweetness,Bitterness,Enjoy,Pairing,Cocktails,Bottom_Line,Points,Price
0,Mezcal de Leyendas Verde San Luis Potosi Mezca...,Mezcal de Leyendas Review Verde San Luis Potos...,Blanco Mezcal,Mexico,45%,"Clear color. Aromas and flavors of cucumber, m...","Fruity, Funky, Rich, Spicy, Smoky, Herbal & Co...","cucumber-melon, lime spritz, caramelized orang...","Same as aromas with notes of pickled ginger, s...",NaN,NaN,"in cocktails, on the rocks and neat",NaN,"Paloma, Sangrita, Tommy's Margarita","A sweet, savory, salty, juicy display of compl...",97 Points,$49.99
1,El Tesoro Paradiso Tequila Extra Anejo Mexico ...,El Tesoro Review Paradiso Tequila Extra Anejo ...,Extra Añejo Tequila,Mexico,40%,"Gold color. Inviting aromas of roasted nuts, c...","Fruity, Complex, Candied, Herbal, Rich & Spicy","roasted nuts, candied cherries, vanilla bean, ...","ripe melons, toffee, and white pepper",NaN,NaN,"in cocktails, neat, on the rocks and with cigars",NaN,"Paloma, Sangrita, Tommy's Margarita",A tantalizing melange of complex flavors that ...,97 Points,$180
2,Peloton de la Muerte Vegan Pechuga Mezcal Mexi...,Peloton de la Muerte Review Vegan Pechuga Mezc...,Flavored Mezcal,Mexico,45.10%,Silvery straw color. Citrusy aromas and flavor...,"Complex, Fruity & Herbal","pickled melon and pineapple, candied jalapeño,...",Same as aromas with suggestions of curry,NaN,NaN,"in cocktails, neat and on the rocks",NaN,"Paloma, Sangrita, Tommy's Margarita","A joyous, extroverted Mezcal with juicy fruit ...",96 Points,$44.99
3,El Tesoro Anejo Tequila Mexico Spirits Review ...,El Tesoro Review Anejo Tequila Mexico Mexican ...,Añejo Tequila,Mexico,40%,Straw color. Inviting aromas and flavors of ho...,"Fruity, Spicy & Herbal","honey-drizzled grapefruit, sweet apple compote...","Same as aromas with notes of pink peppercorns,...",NaN,NaN,"in cocktails, neat and on the rocks",NaN,"Paloma, Sangrita, Tommy's Margarita","A stylish, peppery tequila with an authentic v...",96 Points,$85
4,1800 Milenio Extra Anejo Tequila Mexico Spirit...,1800 Review Milenio Extra Anejo Tequila Mexico...,Extra Añejo Tequila,Mexico,40%,"Amber color. Aromas of toffee coated popcorn, ...","Fruity, Rich & Spicy","toffee coated popcorn, peanuts, warm madeira c...","allspice, mint tea, candied ginger, honey, and...",NaN,NaN,neat and on the rocks,NaN,NaN,A richly flavored Extra Añejo Tequila with an ...,96 Points,$250


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10522 entries, 0 to 10521
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         10517 non-null  object
 1   Description  10517 non-null  object
 2   Category     9467 non-null   object
 3   Country      9461 non-null   object
 4   Alcohol_Vol  9467 non-null   object
 5   Review       9467 non-null   object
 6   Style        9406 non-null   object
 7   Aroma        9429 non-null   object
 8   Flavor       9427 non-null   object
 9   Sweetness    6927 non-null   object
 10  Bitterness   473 non-null    object
 11  Enjoy        9429 non-null   object
 12  Pairing      7171 non-null   object
 13  Cocktails    1643 non-null   object
 14  Bottom_Line  9467 non-null   object
 15  Points       7949 non-null   object
 16  Price        8579 non-null   object
dtypes: object(17)
memory usage: 1.4+ MB


## Review topic modeling

In [31]:
import random

random.seed(42)
reviews = df['Review'].fillna(value='')


In [54]:
#lowercase
reviews = reviews.apply(lambda x: x.lower())

#Remove puntuations
reviews = reviews.apply(lambda x: re.sub(r'[^\w\s]','',x))

#Remove numbers
reviews = reviews.apply(lambda x: re.sub(r'\d+', '', x))

In [55]:
for review in reviews.sample(5).values:
    print(review)
    print('\n')

silver color aromas of peach compote and green apple candy with a satiny crisp spritzy offdry light body and a smooth breezy raspberry slushie finish with no oak flavor a bright and fruity seltzerlike refresher for the summer months


clear dark ruby color earthy aromas and flavors of shredded beet and rhubarb salad dill charcoal tomate de arbol star anise and mixed peppercorns with a satiny vibrant dry medium body and a tingling mediumlong finish that exhibits accents of marinara and toast with crunchy tannins and moderate oak flavor a savory earthy chianti with shining acidity to stand up to robust hearty meals


straw green color aromas of pineapple mango peach diced green pepper and lanolin with a supple crisp dryyetfruity light body and a smooth brisk kiwi and lime finish with no oak flavor a crisp sauvignon blanc with refreshing notes of citrus


deep purple color aromas and flavors of blueberry compote hints of herbs and baking spices with a supple bright dryyetfruity lighttomed

In [27]:
sw = ['color', 'colors', 'aroma', 'aromas', 'flavor', 'flavors']
stop_words = ENGLISH_STOP_WORDS.union(sw)

In [24]:
len(ENGLISH_STOP_WORDS)

318

In [57]:
tfidf = TfidfVectorizer(stop_words=stop_words)

reviews_matrix = tfidf.fit_transform(reviews)

vocabs = tfidf.get_feature_names()

In [60]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

### NMF

In [59]:
nmf = NMF(n_components=10)

nmf.fit(reviews_matrix)

NMF(n_components=10)

In [62]:
reviews_matrix = nmf.transform(reviews_matrix)

In [68]:
reviews_df = pd.DataFrame(reviews_matrix).add_prefix('topic_')
reviews_df['Review'] = reviews
reviews_df.head(10)

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,Review
0,0.000000,0.021830,0.016173,0.031777,0.000000,0.013539,0.036207,0.000000,0.000000,0.000000,clear color aromas and flavors of cucumber mel...
1,0.000000,0.010161,0.024188,0.064934,0.000000,0.018172,0.022767,0.007691,0.000000,0.000000,gold color inviting aromas of roasted nuts can...
2,0.000000,0.004899,0.008995,0.014389,0.020789,0.080346,0.000000,0.010406,0.000000,0.000742,silvery straw color citrusy aromas and flavors...
3,0.006673,0.011876,0.000000,0.043415,0.000000,0.020694,0.047288,0.000000,0.000000,0.021299,straw color inviting aromas and flavors of hon...
4,0.000000,0.002528,0.001464,0.074491,0.000000,0.004351,0.007705,0.003336,0.000000,0.000000,amber color aromas of toffee coated popcorn pe...
5,0.000000,0.000000,0.000000,0.044555,0.006012,0.015866,0.012878,0.038094,0.000000,0.000000,clear color savory aromas and flavors of marin...
6,0.000000,0.000000,0.007281,0.041471,0.000000,0.003735,0.042928,0.021372,0.000000,0.000000,clear color savory aromas of sweet chili sauce...
7,0.011994,0.002214,0.000000,0.026669,0.000000,0.008058,0.014487,0.032559,0.000000,0.000000,straw yellow color savory aromas and flavors o...
8,0.000000,0.000000,0.000000,0.103402,0.005427,0.003685,0.000000,0.000000,0.095582,0.000000,amber color aromas of molasses roasted figs ho...
9,0.007789,0.010556,0.000000,0.033585,0.027265,0.010337,0.023823,0.000688,0.000000,0.000000,clear color aromas of sweet tea gunpowder gril...


In [72]:
reviews_word_df = pd.DataFrame(nmf.components_.round(5), columns=vocabs).T.add_prefix('topic_')

reviews_word_df.head(10)

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
abbey,0.00013,0.00000,0.00079,0.00116,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000
ability,0.00000,0.00133,0.00000,0.00000,0.00000,0.00023,0.00000,0.00043,0.00000,0.0000
abound,0.00000,0.00038,0.01823,0.00000,0.00000,0.00000,0.00000,0.00186,0.00000,0.0000
abounding,0.00000,0.00154,0.00000,0.00000,0.00000,0.00005,0.00000,0.00120,0.00000,0.0000
absinthe,0.00000,0.00250,0.00000,0.00227,0.00101,0.00009,0.00472,0.00000,0.00000,0.0000
absolute,0.00000,0.00000,0.00000,0.00059,0.00000,0.00087,0.00260,0.00303,0.00592,0.0066
absolutely,0.00000,0.00312,0.00227,0.00816,0.00000,0.00000,0.00376,0.00008,0.00000,0.0000
abstain,0.00000,0.00000,0.00000,0.00214,0.00106,0.00000,0.00000,0.00000,0.00053,0.0000
absurdly,0.00000,0.00000,0.00000,0.00381,0.00000,0.00000,0.00338,0.00000,0.00000,0.0000
abv,0.00000,0.00000,0.00000,0.00488,0.00063,0.00042,0.00000,0.00000,0.00000,0.0000


In [78]:
display_topics(nmf, vocabs, 10)


Topic  0
chardonnay, apple, straw, yellow, pineapple, golden, oak, lighttomedium, lemon, apples

Topic  1
black, moderate, plum, violet, tannins, oak, vibrant, cabernet, tea, mediumfull

Topic  2
fruity, mediumlength, satiny, juicy, raspberry, jam, crisp, smooth, ripe, berry

Topic  3
amber, caramel, finish, complex, mediumlong, whiskey, cocktails, warming, long, spicy

Topic  4
light, supple, dryish, brisk, crisp, dry, soft, smooth, finish, quaffer

Topic  5
green, straw, white, blanc, sauvignon, fresh, lime, grapefruit, emerald, ripe

Topic  6
sweet, peach, moderately, spritzy, wine, orange, candies, moscato, candied, fun

Topic  7
dusty, garnet, dried, cherry, nuts, tannins, ruby, medium, pinot, roasted

Topic  8
chewy, slightly, moderate, purple, deep, mediumtofull, tannins, oak, oaky, crisp

Topic  9
pink, strawberry, coral, watermelon, rose, rosé, candies, cherry, pale, steely


### Topic exploration

#### Topic_0

In [74]:
for review in reviews_df.sort_values(by='topic_0', ascending=False).head(10)['Review'].values:
    print(review)
    print('\n')

golden yellow color inviting aromas and flavors of toasted parker dinner rolls with lemon curd pineapple custard and hints of clove with a satiny lively dryyetfruity lighttomedium body and an even charming mediumlong finish with notes of honeycrisp apple and lemon spritz with wellintegrated coating tannins and moderate oak flavor a nicely balanced chardonnay with bright acidity and toasty oak


golden yellow color inviting aromas and flavors of toasted parker dinner rolls with lemon curd pineapple custard and hints of clove with a satiny lively dryyetfruity lighttomedium body and an even charming mediumlong finish with notes of honeycrisp apple and lemon spritz with wellintegrated coating tannins and moderate oak flavor a nicely balanced chardonnay with bright acidity and toasty oak


golden yellow color aromas of apple pie buttery croissant and roasted pineapple with a bright dryyetfruity lighttomedium body and a silky mediumlength toasty oak caramelcoated popcorn and coconut cream fi

In [84]:
reviews_word_df.sort_values(by='topic_0', ascending=False).head(10).topic_0

chardonnay       1.42657
apple            1.24264
straw            1.20751
yellow           1.20349
pineapple        1.17456
golden           1.16401
oak              0.72121
lighttomedium    0.65939
lemon            0.63541
apples           0.52804
Name: topic_0, dtype: float64

**This is topic about Chardonnay white wine**

#### Topic_1

In [85]:
for review in reviews_df.sort_values(by='topic_1', ascending=False).head(10)['Review'].values:
    print(review)
    print('\n')
reviews_word_df.sort_values(by='topic_1', ascending=False).head(10).topic_1

black violet color aromas of plum tart with vanilla meringue latte menthol and raspberry with a silky crisp fruity medium body and a smooth breezy dark chocolate with almonds black licorice and black tea finish with medium tannins and moderate oak flavor a solid merlot that will round out a meal


black garnet color aromas of black currant apple pastry leather sarsaparilla and dark chocolate with a round vibrant fruity mediumtofull body and a warming interesting mediumlong black licorice balsamic reduction and espresso finish with wellintegrated tannins and moderate oak flavor a delicious and fresh cabernet with wellintegrated flavors and tannins highly recommended


garnet black color inviting aromas of ripe plum black licorice black tea eucalyptus honey and peony with a lightly tannic vibrant dry mediumtofull body and a refreshing mediumlong dark chocolate with almonds blueberry jam and toasty crepes finish with drying tannins and moderate oak flavor a dry complex cabernet that will 

black         1.80789
moderate      0.63169
plum          0.52964
violet        0.52919
tannins       0.48140
oak           0.47689
vibrant       0.46841
cabernet      0.44094
tea           0.41839
mediumfull    0.40946
Name: topic_1, dtype: float64

black -> red wine, plum -> sweet, cabernet -> of course cabernet. This is likely about **sweet Cabernet or Merlot**

#### Topic_2

In [90]:
for review in reviews_df.sort_values(by='topic_2', ascending=False).head(10)['Review'].values:
    print(review)
    print('\n')
reviews_word_df.sort_values(by='topic_2', ascending=False).head(10).topic_2

dark ruby color fruity aromas of fruit punch raspberry cobbler and cola with a supple crisp fruity lighttomedium body and a smooth breezy finish with moderate oak flavor a fruity easydrinking red that will be a hit


dark ruby color fruity aromas of fruit punch raspberry cobbler and cola with a supple crisp fruity lighttomedium body and a smooth breezy finish with moderate oak flavor a fruity easydrinking red that will be a hit


dark ruby color fruity aromas of fruit punch raspberry cobbler and cola with a supple crisp fruity lighttomedium body and a smooth breezy finish with moderate oak flavor a fruity easydrinking red that will be a hit


dark ruby color fruity aromas of fruit punch raspberry cobbler and cola with a supple crisp fruity lighttomedium body and a smooth breezy finish with moderate oak flavor a fruity easydrinking red that will be a hit


ruby color fruity aromas of raspberry jam chocolate cake eucalyptus and orange marmalade with a satiny crisp fruity lighttomedium bo

fruity          2.17641
mediumlength    0.59582
satiny          0.59083
juicy           0.49978
raspberry       0.44226
jam             0.43401
crisp           0.42762
smooth          0.40914
ripe            0.34904
berry           0.29829
Name: topic_2, dtype: float64

Some fruity wine, probably **Merlot** or **Pinot Noir**, both of which are delicious with red meat (ribs, steak). :)

#### Topic_3

In [89]:
for review in reviews_df.sort_values(by='topic_3', ascending=False).head(10)['Review'].values:
    print(review)
    print('\n')
reviews_word_df.sort_values(by='topic_3', ascending=False).head(10).topic_3

gold color aromas of coffee cake honey butter and caramel with a satiny crisp dryish lighttomedium body and a warming interesting mediumlong dark toasted whole grain bread sweet roasted corn toffee candied walnut and caramelized banana finish a toasty roasty irish whiskey that warms the palate


gold color aromas of coffee cake honey butter and caramel with a satiny crisp dryish lighttomedium body and a warming interesting mediumlong dark toasted whole grain bread sweet roasted corn toffee candied walnut and caramelized banana finish a toasty roasty irish whiskey that warms the palate


amber color aromas and flavors of winter spice cookies spice drops peach cobbler stroopwafel herbal tea and toasted peanuts with a velvety vibrant dryish medium body and a warming complex mediumlong finish that shows notes of caramel gelato sarsaparilla and coconut a distinctive and slightly herbal bourbon whiskey that delivers rich complexity


amber color aromas and flavors of winter spice cookies spi

amber         0.79109
caramel       0.52603
finish        0.47911
complex       0.46655
mediumlong    0.45073
whiskey       0.39468
cocktails     0.39443
warming       0.39111
long          0.38472
spicy         0.38302
Name: topic_3, dtype: float64

caramel, whiskey -> **bourbon whiskey** (likely)

#### Topic_4

In [91]:
for review in reviews_df.sort_values(by='topic_4', ascending=False).head(10)['Review'].values:
    print(review)
    print('\n')
reviews_word_df.sort_values(by='topic_4', ascending=False).head(10).topic_4

golden straw color aromas of dried apples cream and jute with a supple bright dry light body and a smooth brisk pickled apricot finish with light oak flavor a light chardonnay to serve with a good chill


pale golden straw color aroma of golden apples with a supple bright dryyetfruity light body and an even brisk lemon spritz finish with no oak flavor a light juicy quaffer for simple sipping


pale golden straw color aroma of golden apples with a supple bright dryyetfruity light body and an even brisk lemon spritz finish with no oak flavor a light juicy quaffer for simple sipping


pale straw color aromas and flavors of apples and bananas with a supple bright dryish light body and a sleek brisk finish with no oak flavor an easy white quaffer for everyday indulging


straw color aromas and flavors of lemon rind underripe apple and buttered toast with a lean soft dry light body and a seamless brisk finish with no oak flavor a simple soft quaffer


light gold color aromas of peach jam on 

light      1.59102
supple     0.94643
dryish     0.72080
brisk      0.68982
crisp      0.62614
dry        0.52932
soft       0.48783
smooth     0.47322
finish     0.41457
quaffer    0.39784
Name: topic_4, dtype: float64

#### Topic_5

In [92]:
for review in reviews_df.sort_values(by='topic_5', ascending=False).head(10)['Review'].values:
    print(review)
    print('\n')
reviews_word_df.sort_values(by='topic_5', ascending=False).head(10).topic_5

white gold color fruity aromas and flavors of ripe kiwi fresh green bell pepper white grapefruit zest and white pepper with a satiny vibrant dryyetfruity lighttomedium body and a smooth delightful mediumlength finish with notes of guava and lime spritz with no oak flavor a juicy intense sauvignon blanc that hits every note


white gold color fruity aromas and flavors of ripe kiwi fresh green bell pepper white grapefruit zest and white pepper with a satiny vibrant dryyetfruity lighttomedium body and a smooth delightful mediumlength finish with notes of guava and lime spritz with no oak flavor a juicy intense sauvignon blanc that hits every note


straw green color aromas of dill pickles yellow tomato green pepper and white grapefruit with a satiny crisp dryyetfruity lighttomedium body and a sleek mediumlength lime and green apple finish an herbal savory sauvignon blanc for pairing


pale straw green color aromas of lemon meringue lime and apple jelly with a satiny vibrant dry lighttomed

green         0.83085
straw         0.81478
white         0.80287
blanc         0.75718
sauvignon     0.70799
fresh         0.66620
lime          0.63959
grapefruit    0.59357
emerald       0.52999
ripe          0.39243
Name: topic_5, dtype: float64

In [96]:
for review in reviews_df.sort_values(by='topic_9', ascending=False).head(10)['Review'].values:
    print(review)
    print('\n')
reviews_word_df.sort_values(by='topic_9', ascending=False).head(10).topic_9

bright pink coral color aromas of underripe raspberry crisp pink apples and fresh herbs with a silky vibrant dryyetfruity lighttomedium body and a smooth mediumlength summer watermelon sliced strawberry pink peppercorn and blood orange finish a tasty and bright rose sparkler with much to love


pink coral color aromas and flavors of ripe strawberry and watermelon candies with a satiny crisp offdry light body and an even brisk finish with no oak flavor an offdry simple but tasty rosé with big notes of watermelon candy


pink coral color aromas and flavors of ripe strawberry and watermelon candies with a satiny crisp offdry light body and an even brisk finish with no oak flavor an offdry simple but tasty rosé with big notes of watermelon candy


pale pink coral color aromas and flavors of blood orange marmalade and ripe strawberry with a satiny crisp effervescent dryish lighttomedium body and a polished breezy finish conveying accents of pink grapefruit with no oak flavor a simple slight

pink          1.62542
strawberry    1.25313
coral         1.08119
watermelon    0.84722
rose          0.79200
rosé          0.70421
candies       0.41166
cherry        0.37282
pale          0.35508
steely        0.31160
Name: topic_9, dtype: float64

## Description topic modeling

In [102]:
def text_cleaning(texts):
    #lowercase
    texts = texts.apply(lambda x: x.lower())

    #Remove puntuations
    texts = texts.apply(lambda x: re.sub(r'[^\w\s]','',x))

    #Remove numbers
    texts = texts.apply(lambda x: re.sub(r'\d+', '', x))
    
    return texts

In [107]:
descriptions = df['Description'].fillna('')

descriptions = text_cleaning(descriptions)

tfidf_des = TfidfVectorizer(stop_words=stop_words)

des_matrix = tfidf_des.fit_transform(descriptions)

vocabs_des = tfidf_des.get_feature_names()

In [110]:
nmf = NMF(n_components=10)

nmf.fit(des_matrix)

des_matrix = nmf.transform(des_matrix)

des_df = pd.DataFrame(des_matrix).add_prefix('topic_')
des_df['Description'] = descriptions
des_df.head(10)

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,Description
0,0.006417,0.050884,0.012315,0.000000,0.001946,0.000000,0.000000,0.000000,0.000005,0.000000,mezcal de leyendas review verde san luis potos...
1,0.005488,0.079571,0.009753,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,el tesoro review paradiso tequila extra anejo ...
2,0.001282,0.049880,0.006791,0.000426,0.000814,0.001663,0.005806,0.002435,0.001770,0.001045,peloton de la muerte review vegan pechuga mezc...
3,0.013598,0.083110,0.000000,0.000000,0.002207,0.000000,0.000000,0.000000,0.000000,0.000000,el tesoro review anejo tequila mexico mexican ...
4,0.002395,0.082996,0.001859,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,review milenio extra anejo tequila mexico mex...
5,0.004167,0.070128,0.002724,0.000000,0.002153,0.000000,0.000133,0.000000,0.002887,0.000000,el tesoro review blanco tequila mexico mexican...
6,0.007579,0.051696,0.008574,0.000000,0.000248,0.000000,0.000000,0.000000,0.000561,0.000000,mezcal de leyendas review cenizo durango mezca...
7,0.003630,0.060784,0.010256,0.000000,0.002320,0.000000,0.000000,0.000000,0.001095,0.000000,el tesoro review extra anejo tequila mexico me...
8,0.000000,0.086471,0.004573,0.000000,0.000000,0.000000,0.000000,0.000000,0.002098,0.000000,jose cuervo review reserva de la familia extra...
9,0.009842,0.050570,0.000000,0.000000,0.001770,0.000000,0.000000,0.000000,0.000000,0.030330,montelobos review ensamble joven mezcal mexico...


#### Topic_0

In [111]:
des_word_df = pd.DataFrame(nmf.components_.round(5), columns=vocabs_des).T.add_prefix('topic_')

In [112]:
for description in des_df.sort_values(by='topic_0', ascending=False).head(10)['Description'].values:
    print(description)
    print('\n')
des_word_df.sort_values(by='topic_0', ascending=False).head(10).topic_0

barefoot review moscato nv ca california usa us wine california sweet wine  sweet wine tasting note flavor description fruity crisp  lively juicy  smooth  non oaky smooth moderately sweet no oak sweet peach fondant pears in syrup and orange blossom honey same as aromas with accents of lemon candies a moscato like sweet white best enjoyed with good chill silver medal december  rating   points inexpensive price cheap low priced american eandj gallo winery


barefoot review moscato nv ca california usa us wine california sweet wine  sweet wine tasting note flavor description fruity crisp  lively juicy  smooth  non oaky smooth moderately sweet no oak sweet peach fondant pears in syrup and orange blossom honey same as aromas with accents of lemon candies a moscato like sweet white best enjoyed with good chill silver medal december  rating   points inexpensive price cheap low priced american eandj gallo winery


barefoot review moscato nv ca california usa us wine california sweet wine  swee

sweet          1.88108
wine           1.33913
california     1.14081
moscato        1.07845
nv             1.07539
eandj          0.89989
gallo          0.89866
barefoot       0.88927
low            0.87229
inexpensive    0.87008
Name: topic_0, dtype: float64

#### Topic_1

In [113]:
for description in des_df.sort_values(by='topic_1', ascending=False).head(10)['Description'].values:
    print(description)
    print('\n')
des_word_df.sort_values(by='topic_1', ascending=False).head(10).topic_1

boulder review straight bourbon whiskey co colorado usa us spirits straight bourbon whiskey  whisky spirits tasting note flavor description spicy warming dry engaging cedar popcorn roasted pecan nougat and banana cake cinnamon danish a bold craft whiskey that offers tasty notes of sweet cinnamon pastry silver medal december  rating   points premium price special occasion spendy american vapor distillery liquor booze hooch dependable spirits producer dependable distillery


capmaker review bourbon whiskey tx texas usa us spirits bourbon whiskey  whisky spirits tasting note flavor description mild warming dry cinnamon dusted pear hints of caramelized nuts dried cherry and honeycomb sourdough dulce de leche and black tea a mild bourbon that will play well in craft cocktails silver medal december  rating   points moderate price affordable economical american dynasty spirits liquor booze hooch dependable spirits producer dependable distillery


chicken cock review kentucky straight bourbon 

spirits       1.92816
whiskey       0.83316
vodka         0.77918
liquor        0.72847
booze         0.72089
hooch         0.72089
distillery    0.61057
whisky        0.60359
gin           0.55755
rum           0.52025
Name: topic_1, dtype: float64

#### Topic_2

In [114]:
for description in des_df.sort_values(by='topic_2', ascending=False).head(10)['Description'].values:
    print(description)
    print('\n')
des_word_df.sort_values(by='topic_2', ascending=False).head(10).topic_2

vineyard  review cabernet sauvignon  ca california usa us wine diamond mountain napa valley cabernet sauvignon  red wine tasting note flavor description fruity juicy  smooth oaky rich  full  spicy  complex warming dry yet fruity heavy oak complex marinated beets and beans cranberry dark chocolate nutmeg and menthol fine leather chocolate gelato and plum a creamy velvety cabernet with appetizing savory elements and fruity center delicious gold medal cellar selection march  rating   points luxury price expensive high priced american vineyard  cellar selection great wine producer great winery


crux winery review gsm red blend  ca california usa us wine russian river valley gsm blend red  red wine tasting note flavor description fruity oaky rich  full spicy  complex  juicy  smooth warming dry yet fruity moderate oak interesting red velvet cake ripe plum and cherry sorbet ripe berries sage spiced apple black tea licorice and black pepper a charming rhone varietal blend with excellent food 

red          1.25438
wine         1.05654
cabernet     0.90894
cellar       0.61606
selection    0.56487
fruity       0.54372
great        0.52528
winery       0.47673
rich         0.47473
blend        0.46112
Name: topic_2, dtype: float64

#### Topic_3

In [115]:
for description in des_df.sort_values(by='topic_3', ascending=False).head(10)['Description'].values:
    print(description)
    print('\n')
des_word_df.sort_values(by='topic_3', ascending=False).head(10).topic_3

love noir review pinot noir  ca california usa us wine california pinot noir  red wine tasting note flavor description fruity juicy  smooth  oaky tingling dryish moderate oak candied cherries cranberry tart maple and french toast same as aromas with notes of charred oak an easy pinot noir for the table silver medal november  rating   points moderate price affordable economical american the wine group


love noir review pinot noir  ca california usa us wine california pinot noir  red wine tasting note flavor description fruity juicy  smooth  oaky tingling dryish moderate oak candied cherries cranberry tart maple and french toast same as aromas with notes of charred oak an easy pinot noir for the table silver medal november  rating   points moderate price affordable economical american the wine group


imagery estate review pinot noir  ca california usa us wine california pinot noir  red wine tasting note flavor description fruity juicy  smooth  oaky smooth fruity a suggestion of oak roa

pinot         2.44163
noir          1.83210
grigio        0.81446
wine          0.55939
smooth        0.31023
california    0.29649
dependable    0.27642
fruity        0.26485
cherry        0.24925
ca            0.23245
Name: topic_3, dtype: float64

#### Topic_6

In [116]:
for description in des_df.sort_values(by='topic_6', ascending=False).head(10)['Description'].values:
    print(description)
    print('\n')
des_word_df.sort_values(by='topic_6', ascending=False).head(10).topic_6

avinodos review sauvignon blanc  ca california usa us wine los carneros sauvignon blanc  white wine tasting note flavor description fruity juicy  smooth crisp  lively  non oaky smooth fruity interesting white grapefruit fresh grass cilantro and yuzu same as aromas with overtones of white peach a vibrant herbal sauvignon blanc california sauvignon blanc for marlborough drinker gold medal june  rating   points premium price special occasion spendy american avinodos wines


kopu review sauvignon blanc  new zealand new zealand wine marlborough sauvignon blanc  white wine tasting note flavor description crisp  lively fruity juicy  smooth  non oaky smooth dry yet fruity no oak interesting golden apple passion fruit lime fresh cut grass shortcake and ripe mango same as aromas with accents of tangerine a juicy tropical and fresh sauvignon blanc that will be summer hit silver medal december  rating   points moderate price affordable economical lake road group


kopu review sauvignon blanc  new 

sauvignon     2.52890
blanc         2.40032
cabernet      0.87054
white         0.79049
wine          0.59202
zealand       0.58436
crisp         0.44835
new           0.42702
lively        0.40363
grapefruit    0.39766
Name: topic_6, dtype: float64

#### Topic_8

In [118]:
for description in des_df.sort_values(by='topic_8', ascending=False).head(10)['Description'].values:
    print(description)
    print('\n')
des_word_df.sort_values(by='topic_8', ascending=False).head(10).topic_8

burleigh brewing review twisted palm ale australia australian beer american style pale ale  ale beer tasting note flavor description hoppy smooth dry amusing dried mango and orange pith lavash caraway and crisp lettuces a light and lager like ipa with balanced bitterness silver medal may  rating  points price burleigh brewing company dependable beer producer dependable brewery


great lakes brewing review burning river pale ale oh ohio usa us beer american style pale ale  ale beer tasting note flavor description malty rich  heavy hoppy  smooth  mild smooth dry yet fruity interesting orange blossom honeycomb on toast and butter roasted corn and nuts same as aromas with accents of toasted lemon scone orange melon salad and radish a superb pale ale with great balance flavor and easy drinking style gold medal may  rating  points price american great lakes brewing great beer producer great brewery


lift bridge beer company review jibe talkin apricot sour ale mn minnesota usa us beer sour a

beer          1.97779
ale           0.86246
brewery       0.76472
brewing       0.72570
ipa           0.52329
lager         0.51232
great         0.42141
stout         0.39850
malty         0.37491
dependable    0.31671
Name: topic_8, dtype: float64

#### Topic_9

In [119]:
for description in des_df.sort_values(by='topic_9', ascending=False).head(10)['Description'].values:
    print(description)
    print('\n')
des_word_df.sort_values(by='topic_9', ascending=False).head(10).topic_9

alamos review red blend  argentina argentine wine mendoza other argentine red  red wine tasting note flavor description fruity juicy  smooth  oaky smooth dryish moderate oak pencil shavings plum preserves black tea and sassafras same as aromas with notes of baking spices a bold and chewy red best with beef silver medal  world value wine challenge top  red wine  and under october  rating   points moderate price affordable economical eandj gallo winery best buy best buy good value


alamos review red blend  argentina argentine wine mendoza other argentine red  red wine tasting note flavor description fruity juicy  smooth  oaky smooth dryish moderate oak pencil shavings plum preserves black tea and sassafras same as aromas with notes of baking spices a bold and chewy red best with beef silver medal  world value wine challenge top  red wine  and under october  rating   points moderate price affordable economical eandj gallo winery best buy best buy good value


ruggeri review argeo nv ital

buy           2.15382
best          2.13803
value         1.24716
good          1.06952
wine          0.53865
affordable    0.47963
economical    0.47941
moderate      0.43518
smooth        0.33068
fruity        0.31695
Name: topic_9, dtype: float64

## Clean numerical features

In [151]:
for _ in range(len(df)):
    try:
        df.Points[_] = df.Points[_].split(" ")[0]
    except:
        pass

In [166]:
try:
    df.Points = pd.to_numeric(df.Points, errors='coerce').fillna(0).astype(np.int64)
except:
    pass

In [171]:
for _ in range(len(df)):
    if df.Points[_] == 0:
        df.Points[_] = np.nan
    else:
        pass

In [173]:
df.Points.value_counts()

87.0    1115
88.0    1113
89.0     967
85.0     841
86.0     832
90.0     814
92.0     686
91.0     578
93.0     520
94.0     274
95.0     120
96.0      53
97.0      26
98.0       8
99.0       2
Name: Points, dtype: int64

In [175]:
for _ in range(len(df)):
    try:
        df.Price[_] = df.Price[_].split("$")[1]
    except:
        pass

try:
    df.Price = pd.to_numeric(df.Price, errors='coerce').fillna(0).astype(np.int64)
except:
    pass

In [177]:
for _ in range(len(df)):
    if df.Price[_] == 0:
        df.Price[_] = np.nan
    else:
        pass

In [181]:
df.Price.value_counts()

14.0     489
9.0      487
19.0     431
12.0     415
15.0     393
        ... 
185.0      1
169.0      1
399.0      1
137.0      1
94.0       1
Name: Price, Length: 148, dtype: int64

In [182]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10522 entries, 0 to 10521
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         10517 non-null  object 
 1   Description  10517 non-null  object 
 2   Category     9467 non-null   object 
 3   Country      9461 non-null   object 
 4   Alcohol_Vol  9467 non-null   object 
 5   Review       9467 non-null   object 
 6   Style        9406 non-null   object 
 7   Aroma        9429 non-null   object 
 8   Flavor       9427 non-null   object 
 9   Sweetness    6927 non-null   object 
 10  Bitterness   473 non-null    object 
 11  Enjoy        9429 non-null   object 
 12  Pairing      7171 non-null   object 
 13  Cocktails    1643 non-null   object 
 14  Bottom_Line  9467 non-null   object 
 15  Points       7949 non-null   float64
 16  Price        8577 non-null   float64
dtypes: float64(2), object(15)
memory usage: 1.7+ MB


In [183]:
df.describe()

,Points,Price
count,7949.000000,8577.000000
mean,88.957982,28.889588
std,2.756745,38.085419
min,85.000000,1.000000
25%,87.000000,12.000000
50%,89.000000,19.000000
75%,91.000000,35.000000
max,99.000000,699.000000
